<a href="https://colab.research.google.com/github/jsroa15/KKBOX/blob/main/Members_data_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
#Loading data

members=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KKBOX/members_v3.csv')

In [ ]:
#Exploring data

members.head()


,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [ ]:
#Some usefull information

print('dataframe # of rows: ', members.shape[0])

print('\nnumber of unique counts per msno: ',members.msno.nunique())

print('\nMissing values')

print(members.isnull().sum())



dataframe # of rows:  6769473

number of unique counts per msno:  6769473

Missing values
msno                            0
city                            0
bd                              0
gender                    4429505
registered_via                  0
registration_init_time          0
dtype: int64


In [ ]:
#Exploring gender with current variables

members.groupby(by='gender').agg('count')

,msno,city,bd,registered_via,registration_init_time
gender,,,,,
female,1144613,1144613,1144613,1144613,1144613
male,1195355,1195355,1195355,1195355,1195355


In [ ]:
#Creating mask to filter NaN values in gender

filt=members.gender.isnull()

#Filtering dataframe

prueba=members[filt]

In [ ]:
#Extracting indexes

indexes=list(prueba.index)


#Based on the number on male and females entries the decision was to impute 50%
#of data with male and the other half with women

#Slicing indexes list for replacing

for_women=indexes[0:2214753]
for_men=indexes[2214753:4429506]

#Replacing in original dataframe

members.loc[for_women,'gender']='female'
members.loc[for_men,'gender']='male'

#Checking missing values

members.gender.isnull().sum()

0

In [ ]:
#Exploring bd column

print(members.bd.value_counts())

print(members.bd.describe())

print(np.percentile(members.bd,99.7))

 0      4540215
 22      112200
 21      110574
 20      110452
 27      102769
         ...   
 919          1
 920          1
 923          1
 925          1
-5            1
Name: bd, Length: 386, dtype: int64
count    6.769473e+06
mean     9.795794e+00
std      1.792590e+01
min     -7.168000e+03
25%      0.000000e+00
50%      0.000000e+00
75%      2.100000e+01
max      2.016000e+03
Name: bd, dtype: float64
64.0


In [ ]:
#How many rows less than 0 and greater

print(members[members['bd']<0].shape[0])
print(members[members['bd']>np.percentile(members.bd,99.7)].shape[0])

274
20283


In [ ]:
#Calculating percentage of 'weird' rows

print(members[members['bd']>np.percentile(members.bd,99.7)].shape[0]/members.shape[0]*100)

0.2996245054821845


In [ ]:
#Filter dataframe with postive ages and less than percentile 99.7

prueba=members[(members['bd']>0)&(members['bd']<=np.percentile(members.bd,99.7))]

df=prueba.groupby(by=['gender','city'])['bd'].agg('mean').reset_index()

df[['bd']]=np.round(df.bd,0)

In [ ]:
#Creating a key to merge with members

df['key']=df.apply(lambda row:(row.gender,row.city),axis=1)
df.head()

,gender,city,bd,key
0,female,1,29.0,"(female, 1)"
1,female,3,27.0,"(female, 3)"
2,female,4,29.0,"(female, 4)"
3,female,5,31.0,"(female, 5)"
4,female,6,28.0,"(female, 6)"


In [ ]:
#Creating a Key to merge with df

members['key']=members.apply(lambda row:(row.gender,row.city),axis=1)
members.head()

#Filtering rows with age=0

ceros=members['bd']==0
df1=members[ceros]

#Extracting original indexes
indexes2=list(df1.index)
df1=df1.merge(df,on='key',how='left')

#Setting up indexes in df1
s=pd.Series(indexes2)
df1.set_index(s,inplace=True)

In [ ]:
#Just to make the operations and test if everything is OK
members2=members.copy()

In [ ]:
#Replacing operation
members2.loc[indexes2,'bd']=df1.bd_y

#Checking

members2.isna().sum()

msno                      0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
key                       0
dtype: int64

In [ ]:
#Back to original members df

members.loc[indexes2,'bd']=df1.bd_y

#Applying to members prior filters

df_members=members[(members['bd']>0)&(members['bd']<=np.percentile(members.bd,99.7))]

#Convert registration_init_time to str and then to datetime

df_members['registration_init_time']=df_members['registration_init_time'].astype('str')
df_members['registration_init_time']=pd.to_datetime(df_members['registration_init_time'])
df_members.info()   #Checking


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6748916 entries, 0 to 6769472
Data columns (total 7 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   msno                    object        
 1   city                    int64         
 2   bd                      float64       
 3   gender                  object        
 4   registered_via          int64         
 5   registration_init_time  datetime64[ns]
 6   key                     object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 411.9+ MB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
#Creating a column to extract month number

df_members['month_init']=df_members.registration_init_time.dt.month
df_members.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,msno,city,bd,gender,registered_via,registration_init_time,key,month_init
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,29.0,female,11,2011-09-11,"(female, 1)",9
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,29.0,female,7,2011-09-14,"(female, 1)",9
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,29.0,female,11,2011-09-15,"(female, 1)",9
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,29.0,female,11,2011-09-15,"(female, 1)",9
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32.0,female,9,2011-09-15,"(female, 6)",9


In [ ]:
#Drop Key

df_members.drop(['key'],axis=1,inplace=True)

#Change data types
to_change=['city','registered_via','month_init']

for i in to_change:
  df_members[i]=df_members[i].astype('str')


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
'''

#Exploratory Data Analysis with Pandas Profiling

#Installing Pandas Profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

#Import function
from pandas_profiling import ProfileReport

'''


'\n\n#Exploratory Data Analysis with Pandas Profiling\n\n#Installing Pandas Profiling\n!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip\n\n#Import function\nfrom pandas_profiling import ProfileReport\n\n'

In [ ]:
#Create report and save it in html
#profile = ProfileReport(df_members,title='report of members',html={'style':{'full_width':True}})


In [ ]:
'''
#Exporting Report

profile.to_notebook_iframe()

'''
#Exporting df_members

df_members.to_csv('/content/drive/MyDrive/Colab Notebooks/KKBOX/df_members.csv')